<a href="https://colab.research.google.com/github/essexgroup31/scrappy-doo/blob/main/Copy_of_Covid19_Twitter_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [189]:
# Install Tweepy
#!pip install tweepy
# Install Matplotlib
#!pip install matplotlib


In [190]:
londonGeocodething = (51.509865,-0.118092,1000)

In [211]:
import re
import tweepy
from tweepy import OAuthHandler
consumer_api_key = 'uQP9N32xpuEIujXsDV2VgcekC'
consumer_api_secret = 'aLWDUV2P718ZoOicHCgpvQ8JUWgaOVcce14iA5oXGdtb0PsEV8' 
access_token = '1315962347836579840-gJhPtLkXpchHCz9gMXzYMl6OX9J8Cs'
access_token_sv4ld0F6ecret ='iE0aTCnoKihUv9RkQmAVYYZMeCYh2yT9TprbIG'
print (consumer_api_key)
print (access_token)

uQP9N32xpuEIujXsDV2VgcekC
1315962347836579840-gJhPtLkXpchHCz9gMXzYMl6OX9J8Cs


In [212]:
authorizer = OAuthHandler(consumer_api_key, consumer_api_secret)
authorizer.set_access_token(access_token, access_token_secret)

In [213]:
#search_query = input("Hello user, what would you like to search?\nSearch: ")

In [234]:
api = tweepy.API(authorizer ,timeout=15)
all_tweets = []
ukTweetList = []
search_query = 'covid19'

for tweet_object in tweepy.Cursor(api.search,q=search_query+" -filter:retweets",lang='en',result_type='recent').items(2000):
    all_tweets.append(tweet_object.text)
    #print(tweet_object.user.location)
    if ("UK" or "United Kingdom") in tweet_object.user.location:
      ukTweetList.append(tweet_object)


In [236]:
ukCounties = ["Bedford","Berk","Bristol","Buckingham","Cambridge","Ches","Cornwall","Cumbria","Derby","Devon","Dorset","Durham","Sussex","Essex","Gloucester","Greater London", "Greater Manchester", "Hamp","Hereford","Hertford","Isle of Wight", "Kent", "Lanca", "Leicester","Lincoln","City of London", "Merseyside", "Norfolk", "Northampton", "Northumberland", "North York", "Nottingham", "Oxford", "Rutland", "Shrop", "Somerset","South York", "Staffordshire", "Suffold", "Surrey", "Tyne and Wear", "Warwick", "West Midlands", "West Sussex", "West York", "Wilt", "Worcester"]
# List of uk counties without "shire" at the end (because of the way that twitter works with locations)
for each in ukTweetList:
    location = each.user.location

    for each in range(len(location)):
      if location[each] == ",":
        if location[each+1] != " ":
          location.replace(",",", ")
    
    noComma = False
    while noComma == False:
      if "," in location:
          noComma = False
      else:
          noComma = True
    
      for each in range(len(location)):
          if location[each] == ",":
              location = location.replace(",","")
              break 

    print(location)



Bristol UK
Gloucestershire UK
London UK
UK
Leeds UK
Leeds UK
UK Europe Global
UK
Norwich Norfolk UK
UK
Originally from Nottingham UK
Oxford UK
Coulsdon Surrey UK
UK
UK
13 locations across the UK
YorkshireUK
Stockton-on-Tees UK
London UK
UK 
UK
UK
Raleigh NC and London UK
UK
Newcastle upon Tyne UK
Loughborough UK
UK
London UK
BB9 Lancashire UK
Finchampstead UK
UK
South Tyneside UK
UK
UK
West Midlands UK
UK


In [ ]:
import numpy as np 
import pandas as pd 
import re  
import nltk # an amazing library to play with natural language
nltk.download('stopwords')
from nltk.corpus import stopwords


In [ ]:
tweets = pd.read_csv("https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv")
print(tweets)

In [ ]:
X = tweets.iloc[:, 10].values
y = tweets.iloc[:, 1].values

print(X)

In [ ]:
processed_tweets = []
 
for tweet in range(0, len(X)):  
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', str(X[tweet]))
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    processed_tweets.append(processed_tweet)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(processed_tweets).toarray()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=1)  
text_classifier.fit(X, y)

In [ ]:
negativeCounter = 0
neutralCounter = 0
positiveCounter = 0
for tweet in all_tweets:
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', tweet)
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    sentiment = text_classifier.predict(tfidfconverter.transform([ processed_tweet]).toarray())
    

    # We need to create something that counts the different sentiments
    # Which will then be the data for the pie chart.

    if sentiment == "positive":
      positiveCounter+=1
    elif sentiment == "negative":
      negativeCounter+=1
    elif sentiment == "neutral":
      neutralCounter+=1
   
    print(tweet, sentiment)


    

#print(all_tweets)
print("Positive sentiments:",positiveCounter)
print("Negative sentiments:",negativeCounter)
print("Neutral sentiments:",neutralCounter)

In [ ]:
import matplotlib.pyplot as pyplot
labels = "Negative", "Neutral", "Positive"
sizes = [negativeCounter,neutralCounter,positiveCounter]
print(sizes)
colors = ["red","yellow","green"]

if sizes[0] > sizes[1] & sizes[0] > sizes[2]:
  explode = (0.1,0,0)
elif sizes[1] > sizes[0] & sizes[1] > sizes[2]:
  explode = (0,0.1,0)
else:
  explode = (0,0,0.1)

pyplot.pie(sizes,explode = explode, labels=labels,colors=colors,autopct="%1.1f%%", shadow=True, startangle=140)
pyplot.axis("equal")
pyplot.show()
